In [1]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import sys, os

from lammpsrun import LAMMPS, Prism

from ase import Atoms, units
from ase.visualize import view
from ase.io.trajectory import Trajectory
from ase.io import write, read
from ase.neighborlist import neighbor_list
from ase.build import surface
from ase.spacegroup import crystal
from ase.geometry import *

from pymatgen.core.surface import *
from pymatgen.io.ase import AseAtomsAdaptor as AAA

hpc_path = "G:\\home\\LAMMPS_Simulation\\HPC_Jupyter\\"

# for running on Linux
# hpc_path = "/local/yiming/Desktop/yx6015/home/LAMMPS_Simulation/HPC_Jupyter/"

# print(homepath + '\n' + hpc_path)

Crystallographic data is obtained from https://materials.springer.com/isp/crystallographic/docs/sd_1628167

# Bulk siderite crystal

In [15]:
distorted_siderite_raw = AAA.get_structure(read('fe_o_water_fe_o_c_full.extxyz'))

In [16]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

In [17]:
sg = SpacegroupAnalyzer(distorted_siderite_raw, symprec = 0.1)
distorted_siderite = sg.get_conventional_standard_structure()

In [18]:
sg = SpacegroupAnalyzer(distorted_siderite_raw, symprec = 0.01)
very_distorted_siderite = sg.get_conventional_standard_structure()

In [25]:
distorted_siderite.get_space_group_info(), very_distorted_siderite.get_space_group_info()

(('R-3c', 167), ('P1', 1))

# Surfaces/Slabs

pymatgen referenc: https://matgenb.materialsvirtuallab.org/2017/04/03/Slab-generation-and-Wulff-shape.html

In [26]:
def find_top_site(sites_list, atomstring):
    c_indices = []
    c_coord_z = []

    for i, site in enumerate(sites_list):
        if site.species_string == atomstring:
            c_indices.append(i)
            c_coord_z.append(site.z)

        # Ensuring sequence is sorted
        c_indices_z = sorted(zip(c_coord_z, c_indices))
        
    index_to_remove = c_indices_z[-1][1]
    
    return index_to_remove

In [37]:
def gen_surfaces(layers):
    print("Layers:", layers)
    for key in siderite_surface.keys():
        miller_index = [int(x) for x in key]

        slabgen = SlabGenerator(siderite, miller_index,
                                min_slab_size=layers,
                                min_vacuum_size=10,
                                center_slab=True,
                                in_unit_planes=True)

        slab_list = slabgen.get_slabs(bonds={("C", "O"): 1.5},
                                      symmetrize=True)

        siderite_surface[key] = [x for x in slab_list if abs(x.charge) <= 4]

    surface_110_NS = siderite_surface['110'][2].copy()
    surface_110_NS.symmetrically_remove_atoms([find_top_site(surface_110_NS.sites, 'C4+')])
    surface_012_NS = siderite_surface['012'][2].copy()
    surface_012_NS.symmetrically_remove_atoms([find_top_site(surface_012_NS.sites, 'O2-')])
    surface_012_S = siderite_surface['012'][0].copy()
    surface_012_S_top_C = find_top_site(surface_012_S.sites, 'C4+')
    surface_012_S_top_O3 = surface_012_S.get_neighbors(surface_012_S.sites[surface_012_S_top_C],1.5, include_index=True)
    surface_012_S_top_CO3 = [surface_012_S_top_C] + [x[2] for x in surface_012_S_top_O3]
    surface_012_S.symmetrically_remove_atoms(surface_012_S_top_CO3)

    siderite_surface_kept = {}
    siderite_surface_kept['104_S'] = siderite_surface['104'][0].copy()
    siderite_surface_kept['110_S'] = siderite_surface['110'][1].copy()
    siderite_surface_kept['110_NS'] = surface_110_NS
    siderite_surface_kept['110_NS'] = siderite_surface['110'][0].copy()
    siderite_surface_kept['101_NS0'] = siderite_surface['101'][0].copy()
    siderite_surface_kept['101_NS1'] = siderite_surface['101'][1].copy()
    siderite_surface_kept['001_NS'] = siderite_surface['001'][0].copy()
    siderite_surface_kept['012_S'] = surface_012_S
    siderite_surface_kept['012_NS'] = surface_012_NS
    siderite_surface_kept['100_S'] = siderite_surface['100'][0].copy()

    directory_path = os.path.join(hpc_path, "siderite", "very_distorted_surfaces")
    for surface, slab in siderite_surface_kept.items():
        slab_ase = AAA.get_atoms(slab)
        layer_info, layer_distances = get_layers(slab_ase, (0,0,1), tolerance=0.1)
        nlayers = max(layer_info)

#         print("Surface:", surface, "\tLayers:", nlayers)
        file_path = os.path.join(directory_path, "surface{0}_L{1}.extxyz".format(surface, nlayers))
        slab_ase.write(file_path, format='extxyz')
    return siderite_surface_kept

Past work done with surface 104, 110, 100, 101, 001 and 012. High surface energy on 100 and 101

From bulk optimization we have new cell parameters.

In [38]:
siderite = very_distorted_siderite.copy()
siderite.add_oxidation_state_by_element({"Fe": 2, "C": 4, "O": -2})

In [39]:
siderite_surfaces_miller_indices = ['104', '110', '100', '101', '001', '012']
siderite_surface = dict.fromkeys(siderite_surfaces_miller_indices)

In [41]:
kept_surfaces = []
list_of_layers = np.arange(8, 20)
for i in list_of_layers:
    print(i)
    kept_surfaces.append(gen_surfaces(layers=i))

8
Layers: 8


KeyboardInterrupt: 